<a href="https://colab.research.google.com/github/05satyam/AI-ML/blob/main/Talk_On_Naive_RAG_and_Agentic_RAG_and_LLM_Observability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Some key terminologies:**
- LLM (Large Language Model): AI trained on vast datasets to understand and generate human-like text.
- RAG (Retrieval-Augmented Generation): Combines LLMs with retrieval systems for context-aware responses.
- Embeddings: Numerical representation of data for semantic similarity in vector space.
- Fine-Tuning: Customizing a pre-trained model for specific tasks or domains.
- Prompt Engineering: Crafting inputs to elicit desired outputs from LLMs.
- Guardrails: Safety mechanisms to ensure reliable, ethical model outputs.
- Vector Database: Specialized database for storing and querying embeddings.
- Agents: Autonomous units in AI systems capable of decision-making and task execution.

In [33]:
!pip install llama-index llama-index-callbacks-arize-phoenix

  Using cached pypdf-5.1.0-py3-none-any.whl.metadata (7.2 kB)
Using cached pypdf-5.1.0-py3-none-any.whl (297 kB)
  Attempting uninstall: pypdf
    Found existing installation: pypdf 4.3.1
    Uninstalling pypdf-4.3.1:
      Successfully uninstalled pypdf-4.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.113 requires pypdf<5.0.0,>=4.0.1, but you have pypdf 5.1.0 which is incompatible.


In [6]:
!pip install langchain-openai

## NAIVE-RAG

In [3]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')


### Load embedding models, we are using OpenAI Embeddings for easy access!!

In [7]:
# load embedding model
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model = "text-embedding-3-large")

### ETI -> extract--transform--ingest

- Load the data soruce, we can load multiple documents, here we are using attention is all you need pdf

In [8]:
# load data
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('attention.pdf')
docs=loader.load()

### Text splitter with a default chunk size of 1000, overlap 200
It is always best to experiment with these more.

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents[:5]

[Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing 

### Data Ingestion

In [10]:
#Chroma DB - Opensource, local db setup

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents,OpenAIEmbeddings())

In [11]:
query = "Who are the authors of attention is all you need?"
retireved_results=db.similarity_search(query)


In [12]:
print(retireved_results[0].page_content)

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions


In [ ]:
# Pinecone db setup, serverless connection

In [13]:
!pip install pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.9 MB/s eta 0:00:00


In [14]:
from pinecone import Pinecone

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

#### Pinecone config:

Embedding_model: text-embedding-3-large
Metric : cosine
Dimensions : 3072
Cloud : aws
AWS-Region : us-east-1

In [ ]:
from pinecone import  ServerlessSpec
# to create index from local
# pc.create_index(
#         name='demo-index',
#         dimension=3072,
#         metric="cosine",
#         spec=ServerlessSpec(
#             cloud="aws",
#             region="us-east-1"
#         )
#     )

In [15]:
# When index already created form pinecone dashboard

# connect with index
index = pc.Index("demo-index")

In [16]:
# create vectorstore
from langchain.vectorstores import Pinecone
vectorstore = Pinecone.from_documents(
    documents=documents,
    embedding=embeddings,
    index_name="demo-index"
)

#### Retriever

In [17]:
retriever = vectorstore.as_retriever()

### RAG CHAIN

In [18]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0)




In [19]:
# create document chain
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

template = """"
You are a helpful assistant that answers questions based on the provided context.
Use the provided context to answer the question.
Question: {input}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline
rag_chain = (
    {"context": retriever,  "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [20]:
# response
response = rag_chain.invoke("brief on why we need attention heads?")
response

"We need attention heads, specifically multi-head attention, because they allow the model to jointly attend to information from different representation subspaces at different positions. This is beneficial because a single attention head would average the information, which can inhibit the model's ability to capture diverse patterns and relationships in the data. By using multiple attention heads, each head can focus on different parts of the input, enabling the model to learn more complex and nuanced representations. Additionally, despite having multiple heads, the computational cost remains similar to that of single-head attention due to the reduced dimension of each head."

# Agents

## Single AGENT Application

In [5]:
#Crewai documentaion reference's
# !pip install crewai
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.


In [21]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [22]:
from crewai import Agent, Task, Crew

In [23]:
import os
# from utils import get_openai_api_key

os.environ["OPENAI_MODEL_NAME"] = 'gpt-4'

In [24]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

Agent Writer

In [25]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

Agent Editor

In [26]:
#Edit task
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

Creating task

In [27]:
#plan Task
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [28]:
#Write Task
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [29]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

Create your crew of Agents
Pass the tasks to be performed by those agents.
Note: For this simple example, the tasks will be performed sequentially (i.e they are dependent on each other), so the order of the task in the list matters.
verbose=2 allows you to see all the logs of the execution.

In [30]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

In [31]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer.

Final Answer: 

1. Latest Trends, Key Players, and Noteworthy News in AI:
   - Trends: Machine Learning, AI in healthcare, AI in customer service, reinforcement learning, and AI ethics.
   - Key Players: Google, Microsoft, IBM, Amazon, Facebook, OpenAI, DeepMind.
   - News: Google's AI beating human players at Go, Microsoft's AI for healthcare, Facebook's AI research in deep learning, Amazon's AI in Alexa, OpenAI's GPT-3.

2. Target Audience:
   - Tech enthusiasts, AI and ML professionals, students in tech fields, business

In [32]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Intricacies of Artificial Intelligence: An In-Depth Overview 

Artificial Intelligence (AI) has emerged as a game-changing technology, revolutionizing various facets of our lives, from healthcare to customer service. As a confluence of several sub-disciplines, AI holds the potential to amplify human abilities, automate routine tasks, and open up fresh avenues in scientific exploration. 

## Groundbreaking Trends in AI

AI is not a static realm; it's a whirlwind of innovation and pioneering trends. Machine Learning, a subset of AI, employs statistical methods to empower machines to enhance with experience. This key trend is propelling advancements in predictive analytics and data mining. Reinforcement Learning, another burgeoning trend, employs reward-based systems to train models, powering applications like autonomous cars and game-playing bots. 

AI's role in healthcare is a noteworthy trend. From predicting diseases to tailoring treatments, AI is turning into a healthcare game-changer. AI is also redefining customer service, with AI-powered chatbots offering 24/7 support. Amid these trends, concerns around AI ethics, such as data privacy and algorithmic bias, have become prominent.

## The Majors Players in AI

The swift rise of AI can be attributed to key players who are pushing the boundaries of what's feasible. Tech titans like Google, Microsoft, IBM, Amazon, and Facebook are at the forefront, investing heavily in AI research and development. Google's DeepMind made headlines with its AI AlphaGo, which triumphed over human players in the complex game of Go. 

Microsoft's AI initiatives are revolutionizing healthcare, employing AI to predict patient readmissions and personalize care. Amazon's AI powers Alexa, its virtual assistant, while Facebook's deep learning research is enhancing AI capabilities. Not to be overlooked, OpenAI's GPT-3 has set a new bar in natural language processing.

## Recent AI Chronicles: Notable News and Developments

The AI landscape is in constant flux, with new developments reshaping the field. Facebook's advancements in deep learning, Microsoft's applications of AI in healthcare, Google's victory in Go, and Amazon's Alexa are notable news that underline AI's potential. OpenAI's GPT-3, a language prediction model, has also made ripples, heralding a new era in AI capabilities. 

## Demystifying AI: Addressing Complexities and Staying Current

Understanding AI can be challenging, given its breadth and complexity. However, resources like arXiv's research papers and tech blogs provide insights into the intricate world of AI. Staying updated is vital, with platforms like TechCrunch and The Verge offering the latest AI news.

## Looking Forward: AI Predictions and Ethical Considerations

The future of AI looks bright, with predictions of AI becoming increasingly ingrained in our everyday lives. However, along with these advancements, we must address AI ethics. Concerns around data privacy, algorithmic bias, and job displacement are valid and require consideration as we shape AI's future.

In conclusion, AI is a dynamic, transformative technology that's reshaping our world. Both its advancements and ethical implications warrant our attention and engagement. So, let's stay informed and actively participate in the AI dialogue. After all, AI is not just the future; it's the present.

In [ ]:
'''
Try with another llm
Hugging Face (HuggingFaceHub endpoint)
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()
'''

'\nTry with another llm\nHugging Face (HuggingFaceHub endpoint)\nfrom langchain_community.llms import HuggingFaceHub\n\nllm = HuggingFaceHub(\n    repo_id="HuggingFaceH4/zephyr-7b-beta",\n    huggingfacehub_api_token="<HF_TOKEN_HERE>",\n    task="text-generation",\n)\n\n\n'

## Multi-agent

 Agent can delegate its work to another agent which is better suited to do a particular task.


 allow_delegation=False,

In [ ]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

Tools

In [ ]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

In [ ]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

**Different Ways to Give Agents Tools**
1. Agent Level: The Agent can use the Tool(s) on any Task it performs.
2. Task Level: The Agent will only use the Tool(s) when performing that specific Task.

In [ ]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

In [ ]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

In [ ]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [ ]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


In [ ]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

In [ ]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
The customer needs guidance on how to set up a Crew and specifically how to add memory to the Crew. I need to ensure that I provide a full answer to their question. Since I don't have this information readily available, I will need to use the "Read website content" tool to read the relevant content from the crewAI documentation.

Action: 
Read website content

Action Input: 
{
  "url": "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
}
 

Int

In [ ]:
from IPython.display import Markdown
Markdown(result)

Dear Andrew,

Thank you for reaching out to us. CrewAI is a powerful platform that allows you to create a group of AI agents, each with specific roles, tools, and goals to accomplish complex tasks. This group is what we call a "Crew". 

You've asked about how to add memory to a Crew. I'm sorry for the oversight in my previous response. Let me provide you with the specific steps you can follow:

1. Navigate to the Crews page in your CrewAI dashboard.
2. Click on the specific Crew you want to add memory to.
3. On the Crew details page, find the "Memory" section.
4. Click on the "Add Memory" button.
5. Input the amount of memory you want to add.
6. Confirm your changes by clicking on the "Update" button.

These instructions are also available in our documentation. Please go to the "Managing Crews" section, and look for the subsection titled "Adding Memory to a Crew". 

If you need further assistance or have any other queries, please don't hesitate to ask. We're here to help!

Best regards,
[Your Name]
Senior Support Representative, crewAI

In [ ]:
!pip install llama-index llama-index-callbacks-arize-phoenix

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.5/225.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/9

LLM-Observability

In [ ]:
import os
import llama_index.core
from llama_index.core import set_global_handler



PHOENIX_API_KEY = "87a00ca31b3bee4b9af:ff81696"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
llama_index.core.set_global_handler(
    "arize_phoenix", endpoint="https://llamatrace.com/v1/traces"
)


In [ ]:
from llama_index.llms.openai import OpenAI
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [ ]:
from llama_index.core import VectorStoreIndex, Settings, Document

import openai

llm = OpenAI(model="gpt-4o", temperature=0)

Settings.llm = llm

In [ ]:
documents = [
    Document(text="One Piece does exist!! ~ Whitebeard"),
    Document(text="LlamaIndex helps you build structured applications on top of LLMs."),
    Document(text="Arize Phoenix provides observability for machine learning and LLM applications.")
]
index = VectorStoreIndex.from_documents(documents)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
question = "one piece"
response = query_engine.query(question)
print(response)

One Piece is confirmed to exist by Whitebeard.
